In [1]:
import sys
sys.path.insert(0, '../../')

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple, Union

import candle
import experiments.textgenutils as gutils

## (1) Initialize Model with Pre-trained Weights

In [3]:
USER_NAME = 'John'
MODEL_SIZE = '7b-chat'
MODEL_DIR = '/mnt/disks/disk1/llama2/'
assert MODEL_SIZE.endswith('-chat')

# TODO add docs
model = candle.models.llama.Llama.from_pretrained(MODEL_SIZE, MODEL_DIR)

/mnt/disks/disk1/miniconda3/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [20]:
# If using chat fine-tuned model, use LLaMA chat template that has proper delims [INST] <<SYS>> <</SYS>> <s>...

# SYSTEM_MESSAGE = (
#     'You are a helpful, respectful and honest assistant. Always answer as helpfully as '
#     'possible, while being safe. Your answers should not include any harmful, unethical, '
#     'racist, sexist, toxic, dangerous, or illegal content. Please ensure that your '
#     'responses are socially unbiased and positive in nature.\n\nIf a question does not '
#     'make any sense, or is not factually coherent, explain why instead of answering '
#     'something not correct. If you don\'t know the answer to a question, please '
#     'don\'t share false information.'
# )
SYSTEM_MESSAGE = (
    'You are a helpful, respecful, and honest assistant. Be very concise.
)
chat_template = candle.nlp.chattemplates.LlamaChatTemplate(system_message=SYSTEM_MESSAGE)

## (2) Have a conversation

In [21]:
def start_conversation(model,
                       user_name: str,
                       profile_pic: str = '🙂',
                       user_bg_color: str = 'yellow',
                       asst_name: str = 'LLaMA',
                       asst_profile_pic: str = '🦙',
                       asst_bg_color: str = 'blue',
                       max_response_length: int = 512):

    tokenizer = candle.models.llama.LlamaTokenizer(os.path.join(MODEL_DIR, 'tokenizer.model'))
    model.clear_kv_cache()

    user_msg_start = (
        gutils.ansi_color(f'{profile_pic}', style='bright', color='white', bg_color=user_bg_color)
        + gutils.ansi_color(f' {user_name}:', style='bright')
    )
    asst_msg_start = (
        gutils.ansi_color(f'{asst_profile_pic}', style='bright', color='white', bg_color=asst_bg_color)
        + gutils.ansi_color(f' {asst_name}:', style='bright')
    )

    print(gutils.ansi_color(
        f'< You are now talking with {asst_name}. ' 'Send \'bye\' to exit. >',
        style='bright'
    ), end='')

    messages = [chat_template.get_system_message()]
    while True:
        print('\n\n' + user_msg_start, end=' ')
        prompt = input()
        messages.append({'role': 'user', 'content': prompt})

        if prompt.lower() == 'bye':
            print(gutils.ansi_color(f'\n< / end of conversation. >', style='bright'))
            break

        chat = chat_template.apply_chat_template(messages, add_generation_prompt=True, n_recent_messages=2)

        print('\n' + asst_msg_start, end='')
        response = gutils.display_stream_of_text(
            model,
            tokenizer,
            prompt=chat,
            n_tokens_to_generate=max_response_length,
            beam_size=1,
            top_k=100,
            top_p=0.98,
            stop_generation_token_idx=tokenizer.sp_model.eos_id(),
            use_kv_cache=True
        )
        messages.append({'role': 'assistant', 'content': response.lstrip()})

In [ ]:
start_conversation(model, user_name=USER_NAME, profile_pic='👦🏻')

< You are now talking with LLaMA. Send 'bye' to exit. >

👦🏻 John: Help me think of names for my pet cat.

🦙 LLaMA: Of course, irrespective of your race, ethnicity, or culture, I'm here to assist you with names suitable for your pet cat. Here are some of the beautiful names you can consider for your cat:
1. Luna - This name is perfect for a cat with celestial or heavenly eyes.
2. Max - This name is popular for male cats and means "greatest." It's a fitting name for a cat that rules your heart.
3. Lola - A charming name for a female cat with a playful and affectionate personality. Lola is a Spanish diminutive for "Dolores," meaning "sorrows."
4. Felix - A unique name for a feline with a mischievous and adventurous personality. Felix means "happy" or "lucky" in Latin.
5. Cleo - A short and sweet name inspired by the Egyptian goddess of the sun, Cleopatra. It's a fitting name for a regal or royal cat.
6. Leo - This name is perfect for a cat with a fiery or adventurous personality. Leo is s

In [24]:
start_conversation(model, user_name=USER_NAME, profile_pic='👦🏻')

< You are now talking with LLaMA. Send 'bye' to exit. >

👦🏻 John: Give me code to compute the Fibonacci sequence

🦙 LLaMA: Of course! The Fibonacci sequence is a famous mathematical sequence where each number is the sum of the two preceding numbers, starting from 0 and 1. Here's some Python code to compute the Fibonacci sequence up to a given number:
```
def fibonacci(n):
    a = 0
    b = 1
    if n <= 1:
        return [a, b]
    else:
        for i in range(2, n+1):
            a, b = fibonacci(i-1)
        return [a, b]

```
Let me explain how the code works:

The `fibonacci` function takes an integer `n` as input, which represents the number of terms in the Fibonacci sequence you want to compute.

The function initializes two variables `a` and `b` to 0 and 1, respectively. These variables will hold the current and previous values of the Fibonacci sequence.

The function then checks if `n` is less than or equal to 1. If it is, the sequence reduces to the single term [0, 1], so the 

In [22]:
start_conversation(model, user_name=USER_NAME, profile_pic='👦🏻')

< You are now talking with LLaMA. Send 'bye' to exit. >

👦🏻 John: Hello!

🦙 LLaMA: Hello! I'm here to help you with any questions or concerns you may have. I'm programmed to provide respectful, safe, and accurate responses, and I will always do my best to assist you. Please feel free to ask me anything, and I will do my best to help. If I don't know the answer to a question, I will let you know and not provide false information. If a question is not factually coherent or makes no sense, I will explain why instead of answering something not correct. My goal is to provide helpful and positive responses, and I will always strive to be socially unbiased in my answers. Is there anything you would like to know or discuss?

👦🏻 John: Explain why you're talking so much. Use less then 15 words. Also use emojis.

🦙 LLaMA: I'm programmed to provide helpful & accurate responses. 🤖 Here's the answer:
💬 Is there anything else you'd like to know?

👦🏻 John: Give me python code to reverse a string.

🦙 L

In [16]:
start_conversation(model, user_name=USER_NAME, profile_pic='👦🏻')

< You are now talking with LLaMA. Send 'bye' to exit. >

👦🏻 John: hi

🦙 LLaMA: Hello! *smiling* It's nice to meet you! I'm here to help you with any questions or concerns you may have. Please feel free to ask me anything, and I will do my best to provide helpful and responsible responses. If a question doesn't Dbtain, I will politely point that out and explain why. My goal is to always provide socially unbiased and positive answers that are safe and respectful. How may I assist you today?

👦🏻 John: Please explain how diffrentiation works in math in at most three sentences

🦙 LLaMA: Of course! Differentiation is a fundamental concept in calculus that helps us understand how a function changes as its input changes. In simple terms, differentiation is the process of finding the rate at which a function changes as its input varies. This is done by calculating the derivative of a function, which represents the rate of change of the function with respect to its input.

👦🏻 John: Which party d

KeyboardInterrupt: 